In [ ]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317130 sha256=9b395a443e9825a53562fe6b7446ac6022249d4d90fcfad431068577d1221f46
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


In [ ]:
from pyspark.ml.linalg import Vectors
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LogisticRegression
from sklearn.metrics import accuracy_score

spark = SparkSession.builder.appName('emailspam').getOrCreate()
df = spark.read.csv('emails.csv', header = True, inferSchema = True)
df.printSchema()

# initialization source is the lab (3 - Mobile Price Prediction Solution (Assignment solution))

root
 |-- Email No.: string (nullable = true)
 |-- the: integer (nullable = true)
 |-- to: integer (nullable = true)
 |-- for: integer (nullable = true)
 |-- a: integer (nullable = true)
 |-- you: integer (nullable = true)
 |-- hou: integer (nullable = true)
 |-- is: integer (nullable = true)
 |-- this: integer (nullable = true)
 |-- i: integer (nullable = true)
 |-- your: integer (nullable = true)
 |-- we: integer (nullable = true)
 |-- are: integer (nullable = true)
 |-- com: integer (nullable = true)
 |-- please: integer (nullable = true)
 |-- price: integer (nullable = true)
 |-- attached: integer (nullable = true)
 |-- th: integer (nullable = true)
 |-- forward: integer (nullable = true)
 |-- u: integer (nullable = true)
 |-- click: integer (nullable = true)
 |-- unsubscribe: integer (nullable = true)
 |-- pro: integer (nullable = true)
 |-- therefore: integer (nullable = true)
 |-- cc: integer (nullable = true)
 |-- prize: integer (nullable = true)
 |-- hi: integer (nullable = tr

In [ ]:
from pyspark.sql.functions import *
from pyspark.ml.feature import VectorAssembler, StringIndexer

# here we have to take the input parameters which are the 28 columns containing words and their number of appearances in each mail, and label them under the umbrella of 'features'

vectorAssembler = VectorAssembler(inputCols = ['the', 'to', 'for', 'a', 'you', 'hou', 'is',
'this', 'i', 'your', 'we', 'are', 'com', 'please', 'price', 'attached', 'th', 'forward', 'u', 'click', 'unsubscribe',
'pro', 'therefore', 'cc', 'prize', 'hi', 'deadline', 'ur'], outputCol = 'features')
v_spam_df = vectorAssembler.transform(df)
v_spam_df.show(5)

+---------+---+---+---+---+---+---+---+----+---+----+---+---+---+------+-----+--------+---+-------+---+-----+-----------+---+---------+---+-----+---+--------+---+----+--------------------+
|Email No.|the| to|for|  a|you|hou| is|this|  i|your| we|are|com|please|price|attached| th|forward|  u|click|unsubscribe|pro|therefore| cc|prize| hi|deadline| ur|Spam|            features|
+---------+---+---+---+---+---+---+---+----+---+----+---+---+---+------+-----+--------+---+-------+---+-----+-----------+---+---------+---+-----+---+--------+---+----+--------------------+
|  Email 1|  0|  0|  0|  2|  0|  0|  1|   0|  2|   0|  0|  0|  0|     0|    0|       0|  0|      0|  2|    0|          0|  0|        0|  0|    0|  0|       0|  1|   0|(28,[3,6,8,18,27]...|
|  Email 2|  8| 13|  6|102|  1| 27| 13|   0| 61|   0|  9|  4|  3|     2|    0|       1| 15|      3| 53|    0|          0|  4|        0|  3|    0|  4|       0| 24|   0|[8.0,13.0,6.0,102...|
|  Email 3|  0|  0|  0|  8|  0|  0|  0|   0|  8|   0|  

In [ ]:
# the same thing is done here but labeling the output 'Spam' column as label (our y value)
indexer = StringIndexer(inputCol = 'Spam', outputCol = 'label')
i_v_spam_df = indexer.fit(v_spam_df).transform(v_spam_df)
i_v_spam_df.show(5)

+---------+---+---+---+---+---+---+---+----+---+----+---+---+---+------+-----+--------+---+-------+---+-----+-----------+---+---------+---+-----+---+--------+---+----+--------------------+-----+
|Email No.|the| to|for|  a|you|hou| is|this|  i|your| we|are|com|please|price|attached| th|forward|  u|click|unsubscribe|pro|therefore| cc|prize| hi|deadline| ur|Spam|            features|label|
+---------+---+---+---+---+---+---+---+----+---+----+---+---+---+------+-----+--------+---+-------+---+-----+-----------+---+---------+---+-----+---+--------+---+----+--------------------+-----+
|  Email 1|  0|  0|  0|  2|  0|  0|  1|   0|  2|   0|  0|  0|  0|     0|    0|       0|  0|      0|  2|    0|          0|  0|        0|  0|    0|  0|       0|  1|   0|(28,[3,6,8,18,27]...|  0.0|
|  Email 2|  8| 13|  6|102|  1| 27| 13|   0| 61|   0|  9|  4|  3|     2|    0|       1| 15|      3| 53|    0|          0|  4|        0|  3|    0|  4|       0| 24|   0|[8.0,13.0,6.0,102...|  0.0|
|  Email 3|  0|  0|  0|  

In [ ]:
i_v_spam_df.select('Spam','label').groupBy('Spam','label').count().show()
# just making sure this labeling system worked and that there are no stray values

+----+-----+-----+
|Spam|label|count|
+----+-----+-----+
|   0|  0.0| 3672|
|   1|  1.0| 1500|
+----+-----+-----+



In [ ]:
splits = i_v_spam_df.randomSplit([0.8,0.2],1)
train_df = splits[0]
test_df = splits[1]
train_df.count(), test_df.count(), i_v_spam_df.count()

#classic 80(training)/20(testing) split

(4111, 1061, 5172)

In [ ]:
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
layers = [28,14,4,2]
#a = 28 because the other 2 columns contain the output column (spam) and the other is a simple index
# b = 2 because it's either yes or no

mlp = MultilayerPerceptronClassifier(layers = layers, seed = 1)
mlp_model = mlp.fit(train_df)

pred_df = mlp_model.transform(test_df)

In [ ]:
# I had to rename the column 'Email No.' to Email_ID' because the first nomenclature caused an error (AnalysisException: Syntax error in attribute name: Email No..)
import pandas as pd

pred_df = pred_df.withColumnRenamed("Email No.", "Email_ID")

pred_df.select('Email_ID','features','label','rawPrediction','probability','prediction').show(5)

+----------+--------------------+-----+--------------------+--------------------+----------+
|  Email_ID|            features|label|       rawPrediction|         probability|prediction|
+----------+--------------------+-----+--------------------+--------------------+----------+
|Email 1001|[17.0,15.0,21.0,1...|  0.0|[3.51616301978869...|[0.99614472806413...|       0.0|
| Email 101|(28,[0,1,2,3,5,6,...|  0.0|[3.61512673400691...|[0.99701229754307...|       0.0|
|Email 1010|(28,[0,1,2,3,4,5,...|  0.0|[3.62441161842109...|[0.99705476583800...|       0.0|
|Email 1011|(28,[0,1,2,3,4,5,...|  1.0|[-0.2322944321983...|[0.11937392408127...|       1.0|
|Email 1013|(28,[0,1,2,3,4,6,...|  0.0|[3.02610833483281...|[0.99159801833770...|       0.0|
+----------+--------------------+-----+--------------------+--------------------+----------+
only showing top 5 rows



In [ ]:
evaluator = MulticlassClassificationEvaluator(labelCol = 'label', predictionCol = 'prediction', metricName = 'accuracy')
mlpacc = evaluator.evaluate(pred_df)
mlpacc
# metric name is apparently case sensitive so it can't be written 'Accuracy' and has to be 'accuracy'

0.8501413760603205